In [ ]:
from os import lseek
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import matplotlib.pyplot as plt

finviz_url = "https://finviz.com/quote.ashx?t="

tickers = ["AMZN", "FB", "GOOG", "AAPL", "MSFT"]

news_tables = {}

for ticker in tickers:
    url = finviz_url + ticker
    req = Request(url, headers={"User-Agent": "Mozilla/5.0"})
    res = urlopen(req)
    html = BeautifulSoup(res, "html.parser")
    news_table = html.find(id="news-table")
    news_tables[ticker] = news_table

parsed_data = []

for ticker, news_table in news_tables.items():
    for row in news_table.find_all("tr"):
        title = row.a.text
        date_data = row.td.text.split(" ")

        if len(date_data) == 1:
            time = date_data[0]
        else:
            date = date_data[0]
            time = date_data[1]

        parsed_data.append([ticker, date, time, title])

df = pd.DataFrame(parsed_data, columns=["ticker", "date", "time", "title"])
vader = SentimentIntensityAnalyzer()
f = lambda title: vader.polarity_scores(title)["compound"]
df["compound"] = df["title"].apply(f)
df["date"] == pd.to_datetime(df.date).dt.date

plt.figure(figsize=(10, 8))

mean_df = df.groupby(["ticker", "date"]).mean()
mean_df = mean_df.unstack()
mean_df = mean_df.xs("compound", axis="columns").transpose()
mean_df.plot(kind="bar")
plt.show()
